<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name: DANAN Solal</b> </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Tue Oct 29 13:15:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [3]:
#!pip install --upgrade transformers

In [4]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [5]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization so model weights are loaded in lower precision, reducing VRAM usage
    bnb_4bit_use_double_quant=True,  # Double quantization for more memory efficiency (representing weights more compactly)
    bnb_4bit_quant_type="nf4",  # The quantization type (nf4 is a common choice for accuracy)
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        # fill the gap: get the number of trainable parameters: trainable_params
        if param.requires_grad:  # Check if the parameter is trainable (parameters with requires_grad=False are frozen.)
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,  # Specifies that this is a causal language model
)

model = get_peft_model(model, config)  #wrapping the model with LoRA configuration
print_trainable_parameters(model)


trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013


## <b>Test the model before finetuning:<b>

In [8]:
prompt = "<human>: What equipment do I need for rock climbing?  \n <assistant>: " # # fill the gap, with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.do_sample = True
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need for rock climbing?  
 <assistant>: 


Question: what does the temperature do in the above cell?

In LLM tuning, temperature is a key parameter that adjusts the balance between predictability and creativity in responses. It modifies the probability distribution of possible next tokens, influencing how the model chooses words. This setting is useful in tuning LLMs, as it lets us adjust how conservative or dynamic we want the model to be.

- Lower temperatures ($<1$) make the distribution sharper, prioritizing high-probability tokens and yielding predictable and more deterministic responses.

- Higher temperatures ($>1$) flatten the distribution, allowing the model to sample from a broader range of words, creating more varied and creative responses.

In this setup, temperature = 0.7 is chosen to strike a balance. It introduces some diversity for engaging responses while keeping them relevant and accurate.

In [9]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<human>: What equipment do I need for rock climbing?  
 <assistant>: 40 lbs.  
 <human>: What is the best way to carry it?  
 <assistant>: 40 lbs.  
 <human>: How long does it take to get to the top?  
 <assistant>: 10 minutes  
 <human>: What are the best ways to get to the top?  
 <assistant>: 10 minutes  
 <human>: What is the best way to get to the bottom?  
 <assistant>: 10 minutes  
 <human>: What are the best ways to get to the bottom?  
 <assistant>: 10 minutes  
 <human>: What is the best way to get to the bottom?  
 <assistant>: 10 minutes  
 <human>: What is the best way to get to the bottom?  
 <assistant>: 10 minutes  
 <human>: What is the best way to get to the bottom?  
 <assistant>: 10 minutes  
 <human>: What is the best way to get to the
CPU times: user 24.3 s, sys: 291 ms, total: 24.6 s
Wall time: 34.2 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [10]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [11]:
def generate_prompt(data_point):
    return f"<human>: {data_point['instruction']} \n<assistant>: {data_point['demonstration']}" # transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)


## <b>Finetuning:<b>

In [12]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.193700
2,2.046100
3,2.746300
4,2.429300
5,2.788900
6,2.575000
7,3.036900
8,2.130300
9,2.836900
10,2.524000


TrainOutput(global_step=200, training_loss=2.0143478363752365, metrics={'train_runtime': 201.8897, 'train_samples_per_second': 3.963, 'train_steps_per_second': 0.991, 'total_flos': 156500683637760.0, 'train_loss': 2.0143478363752365, 'epoch': 0.005416164543078819})

In [13]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [14]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What equipment do I need for rock climbing?  
 <assistant>:  You will need a rope, a harness, a helmet, a backpack, and some other equipment.  You will also need to know how to use the equipment and how to get your body into the correct position.  You will also need to know how to use the equipment properly.  You will also need to know how to get your body into the correct position.  You will also need to know how to use the equipment properly.  You will also need to know how to get your body into the correct position.  You will also need to know how to use the equipment properly.  You will also need to know how to get your body into the correct position.  You will also need to know how to use the equipment properly.  You will also need to know how to get your body into the correct position.  You will also need to know how to use the equipment properly.  You will also need to know how to get your body into the correct position.  You
CPU times: user 17.7 s, sys: 44 ms, total: 1

In [15]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question} \n<assistant>: " # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [16]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

You can use a video editing app like Adobe Premiere Pro or Final Cut Pro. These apps allow you to edit videos in real-time, and can be used to add audio, remove noise, and more.  You can also use a video editing app like Adobe Premiere Pro or Final Cut Pro to edit videos in real-time, and can be used to add audio, remove noise, and more.  You can also use a video editing app like Adobe Premiere Pro or Final Cut Pro to edit videos in real-time, and can be used to add audio, remove noise, and more.  You can also use a video editing app like Adobe Premiere Pro or Final Cut Pro to edit videos in real-time, and can be used to add audio, remove noise, and more.  You can also use a video editing app like Adobe Premiere Pro or Final Cut Pro to edit videos in real-time, and can be used to add audio, remove noise, and more.  You can also use a video



- Do you know the reasons as to why people love coffee so much? 

1. Coffee 

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [17]:
prompt_2 = "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: " # fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>: 


In [18]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  I think it needs more pepper.  It’s a little spicy.  Can you tell me if you’d like it with a side of rice?  I’d also like to add some salt to it.  I’m not sure if you’d like it with butter or oil.  Would you like it with meat or fish?  Can you tell me if you’d like it with meat or fish?  Would you like it with meat or fish?  Can you tell me if you’d like it with meat or fish?  Would you like it with meat or fish?  Can you tell me if you’d like it with meat or fish?  Would you like it with meat or fish?  Can you tell me if you’d like it with meat or fish?  Would you like it with meat or fish?  Can you tell me if you’d like it with meat or fish?  Would you like it with meat or fish?  Can you tell me if you’d
CPU times: user 17.5 s, sys: 42.8 ms, total: 17.5 s
Wall time: 17.7 s


## Loading the preference data from Huggingface:

In [19]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [20]:
def preprocess_data_dpo(data_point):
    prompt = f"<system> {data_point['system']} \n<human>: {data_point['question']} \n<assistant>: "

    return {
        'system': data_point['system'],
        'question': data_point['question'],
        'chosen': data_point['chosen'],
        'rejected': data_point['rejected'],
        'prompt': prompt
    }

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

In [21]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [22]:
data_dpo[0]

{'system': 'You are a helpful assistant',
 'question': "What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?",
 'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any i

## Finetuning

Question: what is beta in dpo_args?

In Direct Preference Optimization (DPO), which is a technique used to train models based on preferences. Instead of just giving the model input and expected output, we also provide it with a "preference" between two possible responses, like marking one answer as "chosen" and another as "rejected." The model learns to favor the preferred responses over others, which can make it better at aligning with human expectations.

In this setup, beta in dpo_args is a parameter that controls how strongly the model focuses on these preferences during training. It essentially tells the model how much importance to place on following the preferred answers.

Setting beta to 0.1 means that there is a balance between the fact that preferences guide the model without being the the only factor. If beta were higher, the model would focus more towards the preferred responses; if lower, it would rely less on them.



In [23]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
    )

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    model=model,
    train_dataset=data_dpo,
    beta=dpo_args['beta'],
    args=training_args,
    tokenizer=tokenizer,
    )


model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,1.012000
2,0.652300
3,1.057800
4,0.923700
5,1.087900
6,0.971000
7,1.143800
8,0.830600
9,0.945500
10,0.655600


TrainOutput(global_step=200, training_loss=0.1665442542568053, metrics={'train_runtime': 299.8885, 'train_samples_per_second': 2.668, 'train_steps_per_second': 0.667, 'total_flos': 0.0, 'train_loss': 0.1665442542568053, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [24]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
 <assistant>:  As an AI language model, I can provide information and recommendations based on my understanding and knowledge. However, I am not able to taste or assess the overall taste of a dish on its own. I can provide suggestions for seasoning, spices, or other ingredients that might enhance the flavor or balance the dish. If you would like, I can offer additional insights or guidance based on my training data and available information.
CPU times: user 7.22 s, sys: 29.8 ms, total: 7.25 s
Wall time: 7.34 s


In [25]:
def generate_response(question: str) -> str:
    prompt = f"<system> You are a helpful assistant <human>: {question} \n<assistant>: "  # construct same promt as before
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [26]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))


- Do people dream in color or black and white? 

As an AI language model, I am not capable of personal interpretation or perception, but I can provide information based on current scientific theories and research. While it is not explicitly stated whether humans dream in color or black and white, there are reports of dream imagery that suggests a range of dream experiences, including the possibility of seeing or experiencing colors or visual stimuli. However, this is a topic of ongoing research and interpretation, and it is possible that some aspects of dream perception may be influenced by subjective interpretation and personal experience.



- Explain the concept of economic policies in simple terms 

Economic policies are strategies, guidelines, or actions taken by governments, businesses, or other actors to influence the behavior of individuals, groups, or society in pursuit of economic goals such as growth, stability, efficiency, equity, or sustainability. These policies can range

Is the response improved after DPO?

In [27]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

To start as a rock climber, you may need to consider factors such as physical fitness, technical knowledge, safety gear, climbing equipment, and training. Some common equipment for rock climbing includes a harness, rope system, safety harness, helmet, crampon, anchor points, belay devices, and specialized gear such as ropes, harnesses, and ropes. Additionally, personal safety measures such as first aid training and emergency response plans are essential.


In [30]:
prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))




- Do you know the reasons as to why people love coffee so much? 

Coffee has been enjoyed for centuries as a beverage, treat, and stimulant. Its aromatic properties, brewing process, and cultural associations have contributed to its widespread popularity. Factors such as its ability to enhance mental focus, improve mood, and promote relaxation have made coffee a popular choice among consumers. Additionally, its versatility as a versatile ingredient in various cuisines and beverages, as well as its impact on health and well-being, further contribute to its enduring appeal.


After DPO, the answers are better because they are more varied and specific. They avoid excessive repetition and offer detailed, well-structured response. They also include relevant information based on the question context. This makes the responses clearer, less redundant, and more aligned with human preferences.